In [1]:
%cd ~/SSMuLA

/disk2/fli/SSMuLA


In [2]:
%load_ext autoreload
%autoreload 2
%load_ext blackcellmagic

In [16]:
import numpy as np
import pandas as pd
from scipy import stats

In [11]:
# is boosting always better than ridge?
# do that for one-hot only first
df = pd.read_csv("results/mlde/all_df_comb.csv")

In [12]:
df.columns

Index(['encoding', 'model', 'n_sample', 'ft_lib', 'rep', 'all_maxes',
       'all_means', 'top_maxes', 'top_means', 'ndcgs', 'rhos', 'if_truemaxs',
       'truemax_inds', 'n_mut_cutoff', 'lib', 'zs', 'n_top', 'scale_fit'],
      dtype='object')

In [13]:
df.groupby(['encoding', 'model', 'n_sample', 'ft_lib', 'n_mut_cutoff', 'lib', 'zs', 'n_top'])[['all_maxes',
       'all_means', 'top_maxes', 'top_means', 'ndcgs', 'rhos', 'if_truemaxs',
       'truemax_inds']].mean()

all_maxes  \
encoding model    n_sample ft_lib n_mut_cutoff lib    zs                    n_top              
one-hot  boosting 24       57     single       TrpB3A Triad_score           96      0.773954   
                                                                            384     0.773954   
                                                      esm_score             96      0.773954   
                                                                            384     0.773954   
                                                      esmif_score           96      0.773954   
...                                                                                      ...   
         ridge    1920     80000  all          TrpB4  structnmsa-comb_score 384     0.239959   
                           149361 all          GB1    none                  96      0.115774   
                                                                            384     0.115774   
                           159129 all          TrpB4  none                  96      0.189586   
                                                                            384     0.189586   

                                                                                   all_means  \
encoding model    n_sample ft_lib n_mut_cutoff lib    zs                    n_top              
one-hot  boosting 24       57     single       TrpB3A Triad_score           96      0.029854   
                                                                            384     0.029854   
                                                      esm_score             96      0.029854   
                                                                            384     0.029854   
                                                      esmif_score           96      0.029854   
...                                                                                      ...   
         ridge    1920     80000  all          TrpB4  structnmsa-comb_score 384    -0.002011   
                           149361 all          GB1    none                  96      0.008985   
                                                                            384     0.008985   
                           159129 all          TrpB4  none                  96      0.021406   
                                                                            384     0.021406   

                                                                                   top_maxes  \
encoding model    n_sample ft_lib n_mut_cutoff lib    zs                    n_top              
one-hot  boosting 24       57     single       TrpB3A Triad_score           96      0.683962   
                                                                            384     0.828182   
                                                      esm_score             96      0.683962   
                                                                            384     0.828182   
                                                      esmif_score           96      0.683962   
...                                                                                      ...   
         ridge    1920     80000  all          TrpB4  structnmsa-comb_score 384     0.931767   
                           149361 all          GB1    none                  96      0.740464   
                                                                            384     0.857038   
                           159129 all          TrpB4  none                  96      0.821206   
                                                                            384     0.907079   

                                                                                   top_means  \
encoding model    n_sample ft_lib n_mut_cutoff lib    zs                    n_top              
one-hot  boosting 24       57     single       TrpB3A Triad_score           96     -0.017825   
                                                

In [14]:
df.groupby(['model'])[['all_maxes',
       'all_means', 'top_maxes', 'top_means', 'ndcgs', 'rhos', 'if_truemaxs',
       'truemax_inds']].mean()

,all_maxes,all_means,top_maxes,top_means,ndcgs,rhos,if_truemaxs,truemax_inds
model,,,,,,,,
boosting,0.700737,0.088421,0.853174,0.235182,0.962323,0.284955,0.603407,2479.351882
ridge,0.435390,0.038352,0.872933,0.250510,0.963968,0.281244,0.653648,1733.100589


In [15]:
df.groupby(['model'])[['all_maxes',
       'all_means', 'top_maxes', 'top_means', 'ndcgs', 'rhos', 'if_truemaxs',
       'truemax_inds']].std()

,all_maxes,all_means,top_maxes,top_means,ndcgs,rhos,if_truemaxs,truemax_inds
model,,,,,,,,
boosting,0.314649,0.195859,0.263298,0.297861,0.063008,0.225804,0.489191,10214.841884
ridge,0.373220,0.227822,0.256346,0.301535,0.058505,0.234671,0.475807,8133.680843


In [20]:
# List to hold models
models = df['model'].unique()
results = {}

# Perform pairwise t-tests between models for each variable
for i in range(len(models) - 1):
    for j in range(i + 1, len(models)):
        model1, model2 = models[i], models[j]
        model1_data = df[df['model'] == model1]
        model2_data = df[df['model'] == model2]
        comparison_results = {}
        
        # Perform t-tests on all other columns
        for col in ['all_maxes', 'all_means', 'top_maxes', 'top_means', 'ndcgs', 'rhos', 'if_truemaxs', 'truemax_inds']:  # skip 'model' column
            stat, p_value = stats.ttest_ind(model1_data[col], model2_data[col], equal_var=False)
            mean_diff = model1_data[col].mean() - model2_data[col].mean()
            significance = 'ns'  # not significant
            if p_value < 0.05:
                if mean_diff > 0:
                    significance = 'higher'  # model1 is significantly higher
                else:
                    significance = 'lower'   # model1 is significantly lower
            comparison_results[col] = f'{p_value:.3f} ({significance})'
        results[f'{model1} vs {model2}'] = comparison_results

# Convert results to DataFrame for better visualization
comparison_df = pd.DataFrame(results)
print(comparison_df)

             boosting vs ridge
all_maxes       0.000 (higher)
all_means       0.000 (higher)
top_maxes        0.000 (lower)
top_means        0.000 (lower)
ndcgs            0.000 (lower)
rhos            0.000 (higher)
if_truemaxs      0.000 (lower)
truemax_inds    0.000 (higher)


In [4]:
pd.read_csv("results/mlde/vis_5/all_df.csv")

,encoding,model,n_sample,ft_lib,rep,all_maxes,all_means,top_maxes,top_means,ndcgs,rhos,if_truemaxs,truemax_inds,n_mut_cutoff,lib,zs,n_top,scale_fit
0,esm2_t33_650M_UR50D-flatten_site,boosting,384,4000,0,0.840951,0.156936,1.000000,0.499483,0.930975,0.498705,1.0,267.0,all,DHFR,Triad_score,384,max
1,esm2_t33_650M_UR50D-flatten_site,boosting,384,4000,1,0.837710,0.159552,0.996537,0.500002,0.932816,0.480707,0.0,686.0,all,DHFR,Triad_score,384,max
2,esm2_t33_650M_UR50D-flatten_site,boosting,384,4000,2,0.841871,0.158632,0.996537,0.464790,0.901309,0.464723,0.0,618.0,all,DHFR,Triad_score,384,max
3,esm2_t33_650M_UR50D-flatten_site,boosting,384,4000,3,0.812338,0.180189,1.000000,0.474807,0.926957,0.420841,1.0,2.0,all,DHFR,Triad_score,384,max
4,esm2_t33_650M_UR50D-flatten_site,boosting,384,4000,4,0.841787,0.162283,1.000000,0.485674,0.923643,0.430931,1.0,274.0,all,DHFR,Triad_score,384,max
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1817195,one-hot,ridge,576,20000,45,0.402030,-0.046672,0.932034,0.505388,0.988492,0.230378,0.0,99.0,all,TrpB4,structnmsa-comb_score,96,max
1817196,one-hot,ridge,576,20000,46,0.413499,-0.030766,0.711605,0.366225,0.986520,0.226638,0.0,2576.0,all,TrpB4,structnmsa-comb_score,96,max
1817197,one-hot,ridge,576,20000,47,0.392374,-0.011855,0.725586,0.372355,0.987562,0.241174,0.0,369.0,all,TrpB4,structnmsa-comb_score,96,max
1817198,one-hot,ridge,576,20000,48,0.366519,-0.045167,0.735563,0.405292,0.987685,0.246159,0.0,764.0,all,TrpB4,structnmsa-comb_score,96,max


In [8]:
MLDE_COLORS

['#f79646ff',
 '#f9be00',
 '#89bf91',
 '#6db290',
 '#57a490',
 '#44948f',
 '#33858d',
 '#24768b',
 '#1c6689',
 '#215584',
 '#28437b',
 '#666666']

In [4]:
all_mlde = pd.read_csv("results/mlde/vis_2/all_df.csv")
all_mlde

,encoding,model,n_sample,ft_lib,rep,all_maxes,all_means,top_maxes,top_means,ndcgs,rhos,if_truemaxs,truemax_inds,n_mut_cutoff,lib,zs,n_top
0,esm2_t33_650M_UR50D-flatten_site,boosting,384,4000,0,0.840951,0.156936,1.000000,0.499483,0.930975,0.498705,1.0,267.0,all,DHFR,Triad_score,384
1,esm2_t33_650M_UR50D-flatten_site,boosting,384,4000,1,0.837710,0.159552,0.996537,0.500002,0.932816,0.480707,0.0,686.0,all,DHFR,Triad_score,384
2,esm2_t33_650M_UR50D-flatten_site,boosting,384,4000,2,0.841871,0.158632,0.996537,0.464790,0.901309,0.464723,0.0,618.0,all,DHFR,Triad_score,384
3,esm2_t33_650M_UR50D-flatten_site,boosting,384,4000,3,0.812338,0.180189,1.000000,0.474807,0.926957,0.420841,1.0,2.0,all,DHFR,Triad_score,384
4,esm2_t33_650M_UR50D-flatten_site,boosting,384,4000,4,0.841787,0.162283,1.000000,0.485674,0.923643,0.430931,1.0,274.0,all,DHFR,Triad_score,384
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1229995,one-hot,ridge,576,77,45,0.744631,0.004855,0.739434,0.287961,0.978963,0.189986,0.0,13237.0,single,TrpB4,none,96
1229996,one-hot,ridge,576,77,46,0.771967,0.016475,0.749107,0.300604,0.979479,0.188705,0.0,10338.0,single,TrpB4,none,96
1229997,one-hot,ridge,576,77,47,0.697771,0.006323,0.749107,0.333802,0.980334,0.194473,0.0,13049.0,single,TrpB4,none,96
1229998,one-hot,ridge,576,77,48,0.674717,0.013695,0.749107,0.343322,0.980304,0.190111,0.0,8280.0,single,TrpB4,none,96


In [11]:
all_mlde["n_sample"].unique()

array([384,  96, 192, 288, 480, 576])

In [15]:
all_mlde[(all_mlde["lib"] == "GB1") & (all_mlde["zs"] != "none")]

,encoding,model,n_sample,ft_lib,rep,all_maxes,all_means,top_maxes,top_means,ndcgs,rhos,if_truemaxs,truemax_inds,n_mut_cutoff,lib,zs,n_top
15000,esm2_t33_650M_UR50D-flatten_site,boosting,96,80000,0,0.349655,0.012210,0.469335,0.073868,0.719387,0.186224,0.0,43294.0,all,GB1,Triad_score,384
15001,esm2_t33_650M_UR50D-flatten_site,boosting,96,80000,1,0.277713,0.018058,0.599053,0.030745,0.726337,0.313195,0.0,22534.0,all,GB1,Triad_score,384
15002,esm2_t33_650M_UR50D-flatten_site,boosting,96,80000,2,0.447733,0.021174,0.638133,0.033392,0.693642,0.198916,0.0,44789.0,all,GB1,Triad_score,384
15003,esm2_t33_650M_UR50D-flatten_site,boosting,96,80000,3,0.474581,0.019011,0.831130,0.075928,0.745052,0.286197,0.0,13092.0,all,GB1,Triad_score,384
15004,esm2_t33_650M_UR50D-flatten_site,boosting,96,80000,4,0.291792,0.012217,0.638370,0.072541,0.727522,0.239199,0.0,84165.0,all,GB1,Triad_score,384
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,one-hot,ridge,576,77,45,0.364839,0.053442,0.689580,0.293049,0.765977,0.244508,0.0,1252.0,single,GB1,ev_score,96
999996,one-hot,ridge,576,77,46,0.362723,0.053719,0.658760,0.302206,0.763313,0.242045,0.0,2260.0,single,GB1,ev_score,96
999997,one-hot,ridge,576,77,47,0.379232,0.056310,0.825004,0.281949,0.759752,0.238094,0.0,1786.0,single,GB1,ev_score,96
999998,one-hot,ridge,576,77,48,0.378252,0.049833,0.825004,0.258492,0.760291,0.261377,0.0,1696.0,single,GB1,ev_score,96
